In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans


In [5]:
X = pd.read_pickle('../homework1/X.pkl')
y = pd.read_pickle('../homework1/y.pkl')

In [6]:
X.head()

,age,education.num,capital.gain,capital.loss,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,...,sex_Male,native.country_Canada,native.country_El-Salvador,native.country_Germany,native.country_India,native.country_Mexico,native.country_Philippines,native.country_Puerto-Rico,native.country_United-States,native.country_other
0,90,9,0,4356,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,82,9,0,4356,18,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,66,10,0,4356,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,54,4,0,3900,40,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,41,10,0,3900,40,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
kmeans = KMeans(n_clusters=8, random_state=0)
kmeans.fit(X)

KMeans(random_state=0)

In [27]:
pd.DataFrame(kmeans.labels_,columns=['labels']).labels.value_counts()

1    28460
6     1485
0      752
5      635
7      520
3      493
2      159
4       57
Name: labels, dtype: int64

In [15]:
sum(kmeans.labels_)

159

In [17]:
kmeans.labels_.var()

0.004859297329838416

In [34]:
kmeans.cluster_centers_[0].shape

(59,)

In [35]:
X.shape

(32561, 59)

In [29]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=8)
gm.fit(X)

GaussianMixture(n_components=8)

In [31]:
pd.DataFrame(gm.fit_predict(X),columns=['label']).label.value_counts()

2    11631
4     8732
3     5636
0     2638
6     1738
7     1265
5      762
1      159
Name: label, dtype: int64